In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Define the input and target language vocabularies
input_vocab = ['hello', 'how', 'are', 'you']
target_vocab = ['<START>', '<END>','salut', 'comment', 'ca', 'va']

# Define the maximum sequence length for input and target sentences
max_seq_length = 10

# Create dictionaries to map words to indices and vice versa
input_word2idx = {word: idx for idx, word in enumerate(input_vocab)}
input_idx2word = {idx: word for idx, word in enumerate(input_vocab)}
target_word2idx = {word: idx for idx, word in enumerate(target_vocab)}
target_idx2word = {idx: word for idx, word in enumerate(target_vocab)}

# Define the encoder model
encoder_input = Input(shape=(None,))
encoder_embedding = tf.keras.layers.Embedding(len(input_vocab), 256)(encoder_input)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Define the decoder model
decoder_input = Input(shape=(None,))
decoder_embedding = tf.keras.layers.Embedding(len(target_vocab), 256)(decoder_input)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(target_vocab), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Create the model
model = Model([encoder_input, decoder_input], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Generate training data
input_sentences = ['hello how are you', 'how are you']
target_sentences = ['salut comment ca va', 'comment ca va']
encoder_input_data = np.zeros((len(input_sentences), max_seq_length), dtype='float32')
decoder_input_data = np.zeros((len(target_sentences), max_seq_length), dtype='float32')
decoder_target_data = np.zeros((len(target_sentences), max_seq_length), dtype='float32')

for i, (input_sentence, target_sentence) in enumerate(zip(input_sentences, target_sentences)):
    for t, word in enumerate(input_sentence.split()):
        encoder_input_data[i, t] = input_word2idx[word]
    for t, word in enumerate(target_sentence.split()):
        if t == 0:
            decoder_input_data[i, t] = target_word2idx['<START>']
        else:
            decoder_input_data[i, t] = target_word2idx[word]
        decoder_target_data[i, t] = target_word2idx[word]


# Train the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=2, epochs=100)

# Define the inference models
encoder_model = Model(encoder_input, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_input] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Translate a sentence
def translate_sentence(input_sentence):
    input_tokens = [input_word2idx[word] for word in input_sentence.split()]
    input_seq = tf.keras.preprocessing.sequence.pad_sequences([input_tokens], maxlen=max_seq_length, padding='post')
    states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_word2idx['<START>']
    
    stop_condition = False
    translated_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = target_idx2word[sampled_token_index]
        translated_sentence += ' ' + sampled_word
        
        if sampled_word == '<END>' or len(translated_sentence.split()) > max_seq_length:
            stop_condition = True
        
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        
        states_value = [h, c]
        
    return translated_sentence.strip()

# Test the translation function
input_sentence = 'hello how are you'
translation = translate_sentence(input_sentence)
print('Input Sentence:', input_sentence)
print('Translation:', translation)


Epoch 1/100
1/1 [==============================] - 6s 6s/step - loss: 1.7966
Epoch 2/100
1/1 [==============================] - 0s 106ms/step - loss: 1.7062
Epoch 3/100
1/1 [==============================] - 0s 81ms/step - loss: 1.6074
Epoch 4/100
1/1 [==============================] - 0s 85ms/step - loss: 1.4785
Epoch 5/100
1/1 [==============================] - 0s 91ms/step - loss: 1.2971
Epoch 6/100
1/1 [==============================] - 0s 98ms/step - loss: 1.0567
Epoch 7/100
1/1 [==============================] - 0s 86ms/step - loss: 0.8632
Epoch 8/100
1/1 [==============================] - 0s 88ms/step - loss: 0.8974
Epoch 9/100
1/1 [==============================] - 0s 90ms/step - loss: 0.8852
Epoch 10/100
1/1 [==============================] - 0s 93ms/step - loss: 0.7900
Epoch 11/100
1/1 [==============================] - 0s 88ms/step - loss: 0.6897
Epoch 12/100
1/1 [==============================] - 0s 92ms/step - loss: 0.6299
Epoch 13/100
1/1 [==============================] 